In [2]:
import numpy as np
import random
import cv2
from PIL import Image
import os
from io import BytesIO
from tqdm.notebook import tqdm, trange
import time

In [3]:
def blend_image(original_image: Image.Image, blend_image_source: Image.Image, intensity = 0.4):
    """
        Blends the original image with blend_image with intensity
    """
    
    # converts the blend_image to the format of original image
    # because both the image needs to be in the same format
    # same goes for size
    blend_image_source = blend_image_source.convert(original_image.mode) 
    blend_image_source = blend_image_source.resize(original_image.size)
    
    new_image = Image.new(original_image.mode, original_image.size)
    new_image = Image.blend(original_image, blend_image_source, intensity)
    
    return new_image

In [4]:
def pil_to_np(img_pil):
    """
        Converts image from pil Image to numpy array
    """
    ar: np.ndarray = np.array(img_pil)
    if len(ar.shape) == 3:
        """
            Tensor transpose, since in this case tensor is 3D the order of transpose can be different
            In 2D matrix the transpose is only i,j-> j,i but in more than 2D matrix different permutation can be 
            applied
        """
        ar = ar.transpose(2, 0, 1)
    else:
        ar = ar[None, ...]

    return ar.astype(np.float32) / 255.

In [5]:
def np_to_pil(img_np):
    """
        Converts np.ndarray to Image.Image object
    """
    ar = np.clip(img_np * 255, 0, 255).astype(np.uint8)

    if img_np.shape[0] == 1:
        ar = ar[0]
    else:
        ar = ar.transpose(1, 2, 0)

    return Image.fromarray(ar)

In [6]:
def synthesize_salt_pepper(image: Image.Image, amount, salt_vs_pepper):
    """
        Salt and pepper noise is also known as an impulse noise, this noise can be caused by sharp and sudden 
        disturbances in the image signal. gives the appearance of scattered white or black(or both) pixel over
        the image
    """
    img_pil=pil_to_np(image)

    out = img_pil.copy()
    p = amount
    q = salt_vs_pepper
    flipped = np.random.choice([True, False], size=img_pil.shape,
                               p=[p, 1 - p])
    salted = np.random.choice([True, False], size=img_pil.shape,
                              p=[q, 1 - q])
    peppered = ~salted
    out[flipped & salted] = 1
    out[flipped & peppered] = 0.
    noisy = np.clip(out, 0, 1).astype(np.float32)
    return np_to_pil(noisy)

In [7]:
def synthesize_speckle(image,std_l,std_r):

    ## Give PIL, return the noisy PIL

    img_pil=pil_to_np(image)

    mean=0
    std=random.uniform(std_l/255.,std_r/255.)
    gauss=np.random.normal(loc=mean,scale=std,size=img_pil.shape)
    noisy=img_pil+gauss*img_pil
    noisy=np.clip(noisy,0,1).astype(np.float32)

    return np_to_pil(noisy)

In [8]:
def blur_image_v2(img):
    x=np.array(img)
    kernel_size_candidate=[(3,3),(5,5),(7,7)]
    kernel_size=random.sample(kernel_size_candidate,1)[0]
    std=random.uniform(1.,5.)

    #print("The gaussian kernel size: (%d,%d) std: %.2f"%(kernel_size[0],kernel_size[1],std))
    blur=cv2.GaussianBlur(x,kernel_size,std)

    return Image.fromarray(blur.astype(np.uint8))

In [27]:
def synthesize_low_resolution(image: Image.Image):
    """
        Creates a low resolution image from high resolution image
    """
    width, height = image.size 
    new_width = np.random.randint(int(width / 3), int(width / 2) + 1)
    new_height = np.random.randint(int(height / 3), int(height / 2) + 1)
    
    image = image.resize((new_width, new_height), Image.BICUBIC)
    
#     if random.uniform(0, 1) < 0.5:
#         image = image.resize((width, height), Image.NEAREST)
#     else:
    image = image.resize((width, height), Image.BILINEAR)
        
    return image

In [29]:
def online_add_degradation_v2(img):
    task_id = np.random.permutation(2)

    for x in task_id:
        if x == 0 and random.uniform(0,1)<0.7:
            img = blur_image_v2(img)
        if x == 1 and random.uniform(0,1)<0.7:
            flag = random.choice([1, 2, 3])
            if flag == 1:
                img = synthesize_gaussian(img, 5, 50)
            if flag == 2:
                img = synthesize_speckle(img, 5, 50)
            if flag == 3:
                img = synthesize_salt_pepper(img, random.uniform(0, 0.01), random.uniform(0.3, 0.8))
        # if x == 2 and random.uniform(0,1)<0.7:
        #     img=synthesize_low_resolution(img)

        # if x==3 and random.uniform(0,1)<0.7:
        #     img=convertToJpeg(img,random.randint(40,100))

    return img

In [19]:
def synthesize_gaussian(image,std_l,std_r):

    ## Give PIL, return the noisy PIL

    img_pil=pil_to_np(image)

    mean=0
    std=random.uniform(std_l/255.,std_r/255.)
    gauss=np.random.normal(loc=mean,scale=std,size=img_pil.shape)
    noisy=img_pil+gauss
    noisy=np.clip(noisy,0,1).astype(np.float32)

    return np_to_pil(noisy)

In [12]:
def zero_mask(row, col):
    x = np.zeros((row, col, 3))
    mask=Image.fromarray(x).convert("RGB")
    return mask

def irregular_hole_synthesize(img, mask):
    """
        Create holes using scrach paper textures
        Args:
            img: Original Image
            mask: scratch paper texture
    """
    
    img_np = np.array(img).astype('uint8')
    mask = mask.resize(img.size)
    mask = mask.convert(img.mode) 
    mask_np = np.array(mask).astype('uint8')
    mask_np = mask_np / 255
    img_new=img_np * (1 - mask_np) + mask_np * 255


    hole_img=Image.fromarray(img_new.astype('uint8')).convert("RGB")


    return hole_img,mask.convert("L")

In [13]:
def convertToJpeg(im,quality):
    with BytesIO() as f:
        im.save(f, format='JPEG',quality=quality)
        f.seek(0)
        return Image.open(f).convert('RGB')

In [30]:
import glob

old_image_files = sorted(glob.glob("train/reconstructed_images/*.*"))

for file_name in old_image_files:
    old_image = None
    image_file = file_name.split('/')[-1]
    try:
        original_image = Image.open(f"{file_name}")
        old_image = synthesize_low_resolution(online_add_degradation_v2(original_image))
    except FileNotFoundError:
        print(f"File {file_name} not found")
    
    old_image.save("train/old_images/" + image_file)

(406, 500, 3)
(375, 500, 3)
(375, 500, 3)
(480, 354, 3)
(324, 500, 3)
(333, 500, 3)
(375, 500, 3)
(500, 334, 3)
(364, 480, 3)
(285, 380, 3)
(375, 500, 3)
(500, 375, 3)
(500, 336, 3)
(332, 500, 3)
(500, 334, 3)
(335, 500, 3)
(375, 500, 3)
(333, 500, 3)
(500, 486, 3)
(500, 360, 3)
(375, 500, 3)
(500, 332, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(332, 500, 3)
(333, 500, 3)
(335, 500, 3)
(500, 333, 3)
(333, 500, 3)
(375, 500, 3)
(333, 500, 3)
(500, 375, 3)
(356, 500, 3)
(375, 500, 3)
(375, 500, 3)
(333, 500, 3)
(375, 500, 3)
(333, 500, 3)
(334, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 375, 3)
(333, 500, 3)
(333, 500, 3)
(375, 500, 3)
(500, 496, 3)
(335, 500, 3)
(375, 500, 3)
(272, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 333, 3)
(500, 375, 3)
(375, 500, 3)
(500, 332, 3)
(375, 500, 3)
(333, 500, 3)
(357, 500, 3)
(375, 500, 3)
(333, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(339, 500, 3)
(333, 500, 3)
(323, 500, 3)
(500, 

In [24]:
import glob

old_image_files = sorted(glob.glob("val/reconstructed_image/*.*"))

for file_name in old_image_files:
    old_image = None
    image_file = file_name.split('/')[-1]
    try:
        original_image = Image.open(f"{file_name}")
        old_image = synthesize_low_resolution(online_add_degradation_v2(original_image))
    except FileNotFoundError:
        print(f"File {file_name} not found")
    
    old_image.save("val/old_image/" + image_file)

(500, 486, 3)
(335, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 334, 3)
(375, 500, 3)
(332, 500, 3)
(375, 500, 3)
(375, 500, 3)
(333, 500, 3)
(343, 500, 3)
(500, 333, 3)
(333, 500, 3)
(333, 500, 3)
(375, 500, 3)
(332, 500, 3)
(375, 500, 3)
(500, 375, 3)
(334, 500, 3)
(412, 500, 3)
(375, 500, 3)
(375, 500, 3)
(331, 500, 3)
(375, 500, 3)
(335, 500, 3)
(375, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(336, 500, 3)
(333, 500, 3)
(500, 375, 3)
(435, 450, 3)
(375, 500, 3)
(437, 480, 3)
(375, 500, 3)
(375, 500, 3)
(480, 318, 3)
(275, 500, 3)
(375, 500, 3)
(334, 500, 3)
(375, 500, 3)
(353, 500, 3)
(500, 375, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 387, 3)
(375, 500, 3)
(375, 500, 3)
(309, 500, 3)
(319, 500, 3)
(333, 500, 3)
(375, 500, 3)
(500, 333, 3)
(500, 333, 3)
(328, 500, 3)
(332, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(298, 500, 3)
(375, 500, 3)
(400, 400, 3)
(375, 500, 3)
(434, 500, 3)
(375, 500, 3)
(500, 